In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize

In [3]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:20000]
y_train = y_train[:20000]

In [4]:
# Resize the images to 71x71
x_train = [resize(img, (71, 71), anti_aliasing=True) for img in x_train]
x_test = [resize(img, (71, 71), anti_aliasing=True) for img in x_test]

# Converting the resized images to numpy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)

# Normalize pixel values to the range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot vectors
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
# Load the Xception model, pre-trained on ImageNet
xception = Xception(weights='imagenet', include_top=False, input_shape=(71, 71, 3))

In [6]:
# Add new fully connected layers on top
model = keras.Sequential([
    xception,
    GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 22,969,906
Trainable params: 22,915,378
Non-trainable params: 54,528
_________________________________________________________________


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Train the model
history = model.fit(x_train, y_train, batch_size = 32, epochs = 10, validation_data = (x_test, y_test))

Epoch 1/10
625/625 [==============================] - 258s 400ms/step - loss: 0.8713 - accuracy: 0.7271 - val_loss: 1.7371 - val_accuracy: 0.4849
Epoch 2/10
625/625 [==============================] - 247s 395ms/step - loss: 0.4673 - accuracy: 0.8486 - val_loss: 1.0366 - val_accuracy: 0.6812
Epoch 3/10
625/625 [==============================] - 247s 395ms/step - loss: 0.3284 - accuracy: 0.8945 - val_loss: 0.7581 - val_accuracy: 0.7634
Epoch 4/10
625/625 [==============================] - 246s 394ms/step - loss: 0.2463 - accuracy: 0.9219 - val_loss: 0.9925 - val_accuracy: 0.7446
Epoch 5/10
625/625 [==============================] - 246s 394ms/step - loss: 0.2056 - accuracy: 0.9362 - val_loss: 0.6229 - val_accuracy: 0.8118
Epoch 6/10
625/625 [==============================] - 247s 395ms/step - loss: 0.1669 - accuracy: 0.9488 - val_loss: 1.0108 - val_accuracy: 0.7806
Epoch 7/10
625/625 [==============================] - 247s 395ms/step - loss: 0.1426 - accuracy: 0.9552 - val_loss: 0.6795 -

In [10]:
# Evaluate the model on the test set
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.